# Segment SDS Archive based on launch event times

Launch times come from 'PilotStudy_KSC_Rocket_Launches.xlsx'
SDS archives are at D:/ROCKETSEIS/SDS and contain data from wells 6I and 6S and from seismo-acoustic stations
Segmented event waveform files are saved as MiniSEED to D:/Dropbox/DATA/KSC/KSC_Well_Seismoacoustic_Data/Events

## 1. Read launch data into a DataFrame and generate a list of launch times in Python datetime.datetime format

In [2]:
%run header.ipynb
configDict = wrappers.read_config(configdir=paths['CONFIGDIR'], leader='KSC')
SDS_TOP = configDict['general']['SDS_TOP']
EVENT_MSEED_DIR = os.path.join(configDict['general']['PRODUCTS_TOP'], 'EVENTS')
if not os.path.isdir(EVENT_MSEED_DIR):
    os.makedirs(EVENT_MSEED_DIR)

DROPBOX_PROJECT_DIR = os.path.join(paths['DROPBOX_TOP'], 'PROFESSIONAL', 'RESEARCH', '3_Project_Documents', 'NASAprojects', '201602 Rocket Seismology', '202010 KSC Launchpad Erosion')
#RAWDATA_DIR = os.path.join(paths['DROPBOX_TOP'], 'DATA', 'KSC', 'KSC_Well_Seismoacoustic_Data')

#HTML_DIR = './html'
#PLOTS_DIR = HTML_DIR
excel_launches = os.path.join(DROPBOX_PROJECT_DIR, 'PilotStudy_KSC_Rocket_Launches.xlsx')
csv_launches = excel_launches.replace('.xlsx', '.csv')
#launchesDF = pd.read_excel(excel_launches, sheet_name='launches') # header=0, names=None, index_col=None, usecols=None, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=False, parse_dates=False, date_parser=_NoDefault.no_default, date_format=None, thousands=None, decimal='.', comment=None, skipfooter=0, storage_options=None, dtype_backend=_NoDefault.no_default, engine_kwargs=None)
launchesDF = pd.read_csv(csv_launches)
dt_tmp = pd.to_datetime(launchesDF['Date'] + ' ' +  launchesDF['Time'])
launchesDF['Date'] = [pdt.to_pydatetime() for pdt in dt_tmp]
launchesDF.drop(labels='Time', axis=1, inplace=True)
#print(launchesDF)

## 2. For each launch, segment raw SDS data to multi-trace MiniSEED file in Events directory

In [3]:
#import matplotlib.pyplot as plt

def event_sds2miniseed(launchtime, thisSDSobj, EVENT_MSEED_DIR):    
    pretrig = 600
    posttrig = 1200
    mseedfile = os.path.join(EVENT_MSEED_DIR, 'launch_%s_raw.mseed' % launchtime.strftime('%Y%m%dT%H%M%S'))
    if os.path.exists(mseedfile):
        print('%s already exists' % mseedfile)
    else:
        print('%s not found. Segmenting from SDS.' % mseedfile) 
        startt = launchtime - pretrig
        endt = launchtime + posttrig
        thisSDSobj.read(startt, endt, speed=1)
        st = thisSDSobj.stream
        if len(st)>0:
            st.write(mseedfile, 'mseed')
        else:
            print('Got no data')
            mseedfile = None
    return mseedfile

def clean(st):
    for tr in st:
        if tr.stats.network != 'FL':
            continue
        tr.detrend('linear')
        tr.filter('highpass', freq=0.2, corners=2) 
        
def apply_calibration_correction(st):
    # calibration correction

    for tr in st:
        if 'countsPerUnit' in tr.stats:
            continue
        else:
            tr.stats['countsPerUnit'] = 0.0  
            if tr.stats.channel[1]=='D':
                tr.stats.countsPerUnit = 720 # counts/Pa on 1 V FS setting
                if tr.id[:-1] == 'FL.BCHH3.10.HD':
                    if tr.stats.starttime < UTCDateTime(2022,5,26): # Chaparral M25. I had it set to 1 V FS. Should have used 40 V FS. 
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 8e5 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 720 # counts/Pa 
                    else: # Chaparral switched to 40 V FS
                        if tr.id == 'FL.BCHH3.10.HDF':
                            tr.stats.countsPerUnit = 2e4 # counts/Pa
                        else:
                            tr.stats.countsPerUnit = 18 # counts/Pa 
                tr.stats.units = 'Pa'

            elif tr.stats.channel[1]=='H':
                tr.stats.countsPerUnit = 3e8 # counts/(m/s)
                tr.stats.units = 'm/s'
            tr.data = tr.data/tr.stats.countsPerUnit
    


def maxamp(tr):
    return np.max(np.abs(tr.data))

def remove_spikes(st):
    SEISMIC_MAX = 0.1 # m/s
    INFRASOUND_MAX = 3000 # Pa
    SEISMIC_MIN = 1e-9
    INFRASOUND_MIN = 0.01
    
    for tr in st:
        ma = maxamp(tr)
        if tr.stats.units == 'm/s':
            tr.data[tr.data > SEISMIC_MAX] = np.nan
            tr.data[tr.data < -1 * SEISMIC_MAX] = np.nan             
        elif tr.stats.units == 'Pa':
            tr.data[tr.data > INFRASOUND_MAX] = np.nan
            tr.data[tr.data < -1 * INFRASOUND_MAX] = np.nan     

from obspy.signal.trigger import coincidence_trigger
from pprint import pprint
import matplotlib.dates as dates
def detectEvent(st, launchtime):
    trig = coincidence_trigger("recstalta", 3.5, 1, st, 3, sta=2, lta=40)
    best_trig = {}
    best_product = 0
    for this_trig in trig:
        thistime = dates.date2num(this_trig['time'])
        this_product = this_trig['coincidence_sum']*this_trig['duration']
        if this_product > best_product:
            best_trig = this_trig
            best_product = this_product
    pprint(best_trig)
    return best_trig['time']


def add_snr(st, assoctime, threshold=1.5):
    nstime = max([st[0].stats.starttime, assoctime-240])
    netime = min([st[0].stats.endtime, assoctime-60])
    sstime = assoctime
    setime = min([st[0].stats.endtime, assoctime+120])    
    for tr in st:
        tr_noise = tr.copy().trim(starttime=nstime, endtime=netime)
        tr_signal = tr.copy().trim(starttime=sstime, endtime=setime)
        tr.stats['noise'] = np.nanmedian(np.abs(tr_noise.data))
        tr.stats['signal'] = np.nanmedian(np.abs(tr_signal.data))
        tr.stats['snr'] = tr.stats['signal']/tr.stats['noise']



thisSDSobj = SDS.SDSobj(SDS_TOP)

if not 'mseedfile' in launchesDF.columns:
    launchesDF['mseedfile'] = None
if not 'detected' in launchesDF.columns:
    launchesDF['detected'] = None                             
for i, row in launchesDF.iterrows():
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))                         
    if not row['detected']:
        if not row['mseedfile']:
            mseedfile = event_sds2miniseed(launchTime, thisSDSobj, EVENT_MSEED_DIR)
            launchesDF.at[i, 'mseedfile'] = mseedfile
        try:
            st = read(mseedfile)    
            st.merge()
        except:
            st = Stream()
        print('%s: %d channels' % (mseedfile,len(st)))
        
        # Now we can clean, correct, despike, and detect
        SA = Stream()
        for tr in st:
            if tr.stats.network[0] != '6':
                SA.append(tr)
        if len(SA)==0:
            continue
        
        clean(SA)
        apply_calibration_correction(SA)
        remove_spikes(SA)
        print('- %d channels after despiking' % len(SA))
        assocTime = detectEvent(SA, launchTime)

        #look_for_signal(SA, assocTime, threshold=1.1)
        detectedfile =  os.path.join(EVENT_MSEED_DIR, 'launch_%s_detected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
        print('Writing %s' % detectedfile)
        st.write(detectedfile, format='PICKLE')  
        launchesDF.at[i, 'detected'] = assocTime
            
        # loop over stations, plotting all traces
        stations = list(set([tr.stats.station for tr in st]))
        for station in stations:
            station_stream = st.copy().select(station=station)
            station_stream.trim(starttime=assocTime-60, endtime=assocTime+200).plot(equal_scale=False, outfile=detectedfile.replace('.pkl', '_%s.png' % station))
del thisSDSobj
launchesDF.to_csv(csv_launches.replace('.csv','_detected.csv'))          

Processing launch at 2022-04-01 16:24:16
D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220401T162416_raw.mseed not found. Segmenting from SDS.
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD4.D\FL.BCHH2.10.HD4.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD5.D\FL.BCHH2.10.HD5.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD6.D\FL.BCHH2.10.HD6.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD7.D\FL.BCHH2.10.HD7.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD8.D\FL.BCHH2.10.HD8.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH2\HD9.D\FL.BCHH2.10.HD9.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHE.D\FL.BCHH3.00.HHE.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHN.D\FL.BCHH3.00.HHN.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HHZ.D\FL.BCHH3.00.HHZ.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\BCHH3\HDF.D\FL.BCHH3.10.HDF.D.2022.091")
st = read("D:/ROCKETSEIS/SDS\2022\FL\S39A1\HHE.D\FL.S39A1.00.HHE.D.2022.091")

C:\Users\thompsong\Anaconda3\envs\iceweb\lib\site-packages\obspy\io\mseed\core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')
C:\Users\thompsong\Anaconda3\envs\iceweb\lib\site-packages\obspy\io\mseed\core.py:823: UserWarning: File will be written with more than one different record lengths.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'record lengths')


D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220724T133820_raw.mseed: 30 channels
- 20 channels after despiking
{'coincidence_sum': 19.0,
 'duration': 93.12999987602234,
 'similarity': {},
 'stations': ['S39A2',
              'S39A1',
              'S39A1',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A2',
              'BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH4'],
 'time': UTCDateTime(2022, 7, 24, 13, 37, 50, 760000),
 'trace_ids': ['FL.S39A2.10.HDF',
               'FL.S39A1.10.HDF',
               'FL.S39A1.00.HHN',
               'FL.S39A1.00.HHE',
               'FL.S39A2.00.HHN',
               'FL.S39A1.00.HHZ',
               'FL.S39A2.00.HHZ',
               'FL.S39A2.00.HHE',
               'FL.BCHH2.10.HD5',
           

C:\Users\thompsong\Anaconda3\envs\iceweb\lib\site-packages\obspy\io\mseed\core.py:821: UserWarning: File will be written with more than one different encodings.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'encodings')
C:\Users\thompsong\Anaconda3\envs\iceweb\lib\site-packages\obspy\io\mseed\core.py:823: UserWarning: File will be written with more than one different record lengths.
This might have a negative influence on the compatibility with other programs.
  warnings.warn(msg % 'record lengths')


D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220804T102900_raw.mseed: 24 channels
- 20 channels after despiking
{'coincidence_sum': 20.0,
 'duration': 47.40000009536743,
 'similarity': {},
 'stations': ['BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A1',
              'BCHH4',
              'BCHH2'],
 'time': UTCDateTime(2022, 8, 4, 10, 29, 3, 370000),
 'trace_ids': ['FL.BCHH2.10.HD6',
               'FL.BCHH4.00.HDF',
               'FL.BCHH2.10.HD7',
               'FL.BCHH2.10.HD8',
               'FL.BCHH2.10.HD9',
               'FL.BCHH2.10.HD4',
               'FL.BCHH4.00.HD2',
               'FL.BCHH4.10.HHE',
               'FL.BCHH4

...

Processing launch at 2022-06-17 16:09:20
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220617T160920_raw.mseed already exists
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220617T160920_raw.mseed: 20 channels
- 20 channels after despiking
{'coincidence_sum': 18.0,
 'duration': 78.03999996185303,
 'similarity': {},
 'stations': ['S39A2',
              'S39A1',
              'S39A1',
              'S39A1',
              'S39A2',
              'S39A2',
              'BCHH2',
              'S39A1',
              'S39A2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH2'],
 'time': UTCDateTime(2022, 6, 17, 16, 9, 4, 20000),
 'trace_ids': ['FL.S39A2.00.HHN',
               'FL.S39A1.00.HHZ',
               'FL.S39A1.00.HHE',
               'FL.S39A1.00.HHN',
               'FL.S39A2.00.HHE',
         

Processing launch at 2022-07-17 14:20:00
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220717T142000_raw.mseed already exists
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220717T142000_raw.mseed: 20 channels
- 20 channels after despiking
{'coincidence_sum': 18.0,
 'duration': 77.63000011444092,
 'similarity': {},
 'stations': ['BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'S39A2',
              'S39A2',
              'S39A2',
              'S39A1',
              'S39A1',
              'S39A1',
              'BCHH4',
              'S39A1',
              'S39A2'],
 'time': UTCDateTime(2022, 7, 17, 14, 20, 1, 520000),
 'trace_ids': ['FL.BCHH4.10.HHN',
               'FL.BCHH4.10.HHE',
               'FL.BCHH4.10.HHZ',
               'FL.BCHH2.10.HD4',
               'FL.BCHH4.00.HD2',
       

Processing launch at 2022-08-28 03:41:20
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220828T034120_raw.mseed already exists
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20220828T034120_raw.mseed: 26 channels
- 16 channels after despiking
{'coincidence_sum': 15.0,
 'duration': 71.07999992370605,
 'similarity': {},
 'stations': ['BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'S39A1',
              'S39A1',
              'S39A1',
              'S39A1',
              'BCHH4'],
 'time': UTCDateTime(2022, 8, 28, 3, 41, 21, 650000),
 'trace_ids': ['FL.BCHH4.10.HHN',
               'FL.BCHH4.10.HHE',
               'FL.BCHH4.10.HHZ',
               'FL.BCHH2.10.HD7',
               'FL.BCHH2.10.HD9',
               'FL.BCHH4.00.HDF',
               'FL.BCHH2.10.HD8',
        

Processing launch at 2022-10-05 16:00:57
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221005T160057_raw.mseed already exists
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221005T160057_raw.mseed: 30 channels
- 20 channels after despiking
{'coincidence_sum': 17.0,
 'duration': 86.57000017166138,
 'similarity': {},
 'stations': ['S39A1',
              'BCHH4',
              'BCHH2',
              'S39A1',
              'S39A2',
              'S39A1',
              'S39A2',
              'S39A2',
              'S39A1',
              'S39A2',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'BCHH4',
              'BCHH4',
              'BCHH2'],
 'time': UTCDateTime(2022, 10, 5, 16, 0, 32, 210000),
 'trace_ids': ['FL.S39A1.10.HDF',
               'FL.BCHH4.00.HDF',
               'FL.BCHH2.10.HD6',
               'FL.S39A1.00.HHN',
               'FL.S39A2.00.HHN',
               'FL.S39A1.00.HH

Processing launch at 2022-11-12 16:06:00
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221112T160600_raw.mseed already exists
D:/Dropbox\DATA\KSC\KSC_Well_Seismoacoustic_Data\Events\launch_20221112T160600_raw.mseed: 16 channels
- 16 channels after despiking
{'coincidence_sum': 15.0,
 'duration': 68.87000012397766,
 'similarity': {},
 'stations': ['BCHH4',
              'BCHH4',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH4',
              'BCHH2',
              'BCHH2',
              'BCHH2',
              'BCHH4',
              'S39A1',
              'S39A1',
              'BCHH4',
              'S39A1',
              'S39A1'],
 'time': UTCDateTime(2022, 11, 12, 16, 6, 2, 560000),
 'trace_ids': ['FL.BCHH4.10.HHZ',
               'FL.BCHH4.10.HHN',
               'FL.BCHH2.10.HD4',
               'FL.BCHH4.00.HDF',
               'FL.BCHH2.10.HD7',
               'FL.BCHH4.00.HD2',
               'FL.BCHH2.10.HD8',
        

## 3. Generate HTML index

In [4]:
HTML_DIR = os.path.join(configDict['general']['PRODUCTS_TOP'], 'HTML')
if not os.path.isdir(HTML_DIR):
    os.makedirs(HTML_DIR)
indexfile = open(os.path.join(HTML_DIR, "index.html"), "w")
print(f"Writing {indexfile}")
indexfile.write("""
<html>

<head>
<title>Launch data browser</title>
</head>

<body>

<table border=1>

<tr>
<th>Detected launch time</th>
<th>JulDay</th>
<th>epoch</th>
<th>Seismic</th>
<th>Infrasound</th>
<th>Well_6S</th>
<th>Well_6I</th>
</tr>

<tr>
""")


for i, row in launchesDF.iterrows():
    print('\n\n\n')
    launchTime = UTCDateTime(row['Date'])
    print('Processing launch at %s' % launchTime.strftime('%Y-%m-%d %H:%M:%S'))  
    indexfile.write("<td>%s</td>\n<td>%s</td>\n<td>%.0f</td>\n" % (launchTime, launchTime.julday, launchTime.timestamp))
    
    ## Seismic
    detectedfile =  os.path.join(EVENT_MSEED_DIR, 'launch_%s_detected.pkl' % launchTime.strftime('%Y%m%dT%H%M%S'))
    seismicpngfile = os.path.join(HTML_DIR, 'launch_%s_seismic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(seismicpngfile):
        print('Reading %s' % detectedfile)
        try:
            st = read(detectedfile, format='PICKLE')
            if row['detected']:
                dt = row['detected']
                stime = max([st[0].stats.starttime, dt-30])
                etime = min([st[0].stats.endtime, dt+150])
                st.trim(starttime=stime, endtime=etime)
        except:
            st = Stream()
        st_seismic = st.select(channel='HH?').select(network='FL')
        if len(st_seismic)>0:
            st_seismic.plot(equal_scale=False, outfile=seismicpngfile )            
    if os.path.exists(seismicpngfile):
        st_seismic = read(detectedfile, format='PICKLE').select(channel='HH?').select(network='FL')
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(seismicpngfile), len(st_seismic)))
    else:
        indexfile.write("<td>0</td>\n")
        
    ## Acoustic
    acousticpngfile = os.path.join(HTML_DIR, 'launch_%s_infrasound.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    if not os.path.exists(acousticpngfile):
        if not st: # in locals():
            print('Reading %s' % detectedfile)
            try:
                st = read(detectedfile, format='PICKLE')
            except:
                st = Stream()
        st_acoustic = st.select(channel='HD?').select(network='FL')
        
        if len(st_acoustic)>0: 
            st_acoustic.plot(equal_scale=False, outfile=acousticpngfile )
    if os.path.exists(acousticpngfile): 
        st_acoustic = read(detectedfile, format='PICKLE').select(channel='HD?').select(network='FL')
        indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(acousticpngfile), len(st_acoustic) ))
    else:
        indexfile.write("<td>0</td>\n")    
         
    # Well 6S
    for well in ['6S', '6I']:
        wellpngfile = os.path.join(HTML_DIR, 'launch_%s_well_%s.png' % (launchTime.strftime('%Y%m%dT%H%M%S'), well) )
        if not os.path.exists(wellpngfile):
            if not 'st': # in locals():
                print('Reading %s' % detectedfile)
                try:
                    st = obspy.read(detectedfile)
                except:
                    st = Stream()
            st_well = st.select(network=well)
            if len(st_well)>0: 
                st_well.plot(equal_scale=False, outfile=wellpngfile )
        if os.path.exists(wellpngfile):   
            st_well = read(detectedfile, format='PICKLE').select(network=well)
            indexfile.write("<td><a href=\'%s\'>%d</a></td>\n" % (os.path.basename(wellpngfile), len(st_well)))
        else:
            indexfile.write("<td>0</td>\n")              

        
    # comments
    #indexfile.write("<td>%s</td>\n" % launchcomments[launchnum])
    
    # end this row of table
    indexfile.write("</tr>\n\n")

indexfile.write("""

</table>

</body>

</html>
""")
indexfile.close()


Writing <_io.TextIOWrapper name='D:/ROCKETSEIS/ICEWEB_PRODUCTS\\HTML\\index.html' mode='w' encoding='cp1252'>




Processing launch at 2022-04-01 16:24:16
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220401T162416_detected.pkl




Processing launch at 2022-04-08 15:17:12
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220408T151712_detected.pkl




Processing launch at 2022-04-21 17:51:40
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220421T175140_detected.pkl




Processing launch at 2022-04-27 07:52:55
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220427T075255_detected.pkl




Processing launch at 2022-04-29 21:27:10
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220429T212710_detected.pkl




Processing launch at 2022-05-06 09:42:00
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220506T094200_detected.pkl




Processing launch at 2022-05-14 20:40:50
Reading D:/ROCKETSEIS/ICEWEB_PRODUCTS\EVENTS\launch_20220514T204050_detected.pkl




Proc

# 4. Generate event browser

In [5]:
import glob
launchpngs = sorted(glob.glob(os.path.join(HTML_DIR, "*_seismic.png")))
    
htmlstr = '''
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Launch Events</title>

    <!-- Basic style -->
    <style>
        .img-container {
            margin: 20px auto;
            width: 40%%;
        }

        .btn-container {
            text-align: center;
        }

        img {
            width: 600px;
            display: block;
            margin: auto;
            padding-bottom: 20px;
        }

        button {
            outline: none;
            padding: 12px;
            border: none;
            background-color: rgb(25, 121, 211);
            border-radius: 8px;
            color: white;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <!-- HTML code for next and previous button -->
    <div class="img-container">
'''
htmlstr += f"<h1 id='imageTitle'>{os.path.basename(launchpngs[0])} </h1>"
htmlstr += f"<img src='{os.path.basename(launchpngs[0])}' class='imageTag'>"
htmlstr += '''
        <div class="btn-container">
            <button onclick="previous()">Previous</button>
            <button onclick="next()">Next</button>
        </div>
    </div>

    <!-- Javascript code for next and previous button -->
    <script>
        let images = [
'''
for launchpng in launchpngs:
    htmlstr += f"'{os.path.basename(launchpng)}',"
htmlstr += '''
        ];
        let imageTag = document.querySelector('.imageTag');
        var heading = document.getElementById('heading');
    
        let i = 0;

        function next() {
            if (i >= images.length - 1) {
                return false;
            }
            i++;
            imageTag.setAttribute('src', images[i]);
            imageTitle.textContent = images[i];
        }
        function previous() {
            if (i <= 0) {
                return false;
            }
            i--;
            imageTag.setAttribute('src', images[i]);
            imageTitle.textContent = images[i];
        }

    </script>

</body>

</html>
'''

with open(os.path.join(HTML_DIR,"launches.html"), "w") as text_file:
    text_file.write(htmlstr)

Modify above so that it looks for seismic, infrasound, and well data PNGs, and moves them all forward at once.
Perhaps we save everything to a CSV file - all the file names and any titles, descriptors, etc. Then we can load that into a dataframe.
Then within next and previous, we load appropriate row, and update things.
Could also have a clickable index in a separate frame to the left.

In [12]:
print(launchesDF)

                  Date  SLC                                    Rocket_Payload  \
0  2022-04-01 16:24:16   40                  Falcon 9 Block 5 | Transporter 4   
1  2022-04-08 15:17:12  39A  Falcon 9 Block 5 | F9-147 Axiom Mission 1 (Ax-1)   
2  2022-04-21 17:51:40   40            Falcon 9 Block 5 | Starlink Group 4-14   
3  2022-04-27 07:52:55  39A                  Falcon 9 Block 5 | SpaceX Crew-4   
4  2022-04-29 21:27:10   40            Falcon 9 Block 5 | Starlink Group 4-16   
5  2022-05-06 09:42:00  39A            Falcon 9 Block 5 | Starlink Group 4-17   
6  2022-05-14 20:40:50   40            Falcon 9 Block 5 | Starlink Group 4-15   
7  2022-05-18 10:59:40  39A            Falcon 9 Block 5 | Starlink Group 4-18   
8  2022-05-19 22:54:47   41              Atlas V N22 | AV-082 Starliner OFT-2   
9  2022-05-25 18:35:00   40                  Falcon 9 Block 5 | Transporter 5   
10 2022-06-08 21:04:00   40             Falcon 9 Block 5 | F9-157 Nilesat-301   
11 2022-06-12 17:43:00   46 

In [43]:
import glob
launchpngs = sorted(glob.glob(os.path.join(HTML_DIR, "*_seismic.png")))
seismicpngs = []
acousticpngs = []
wellpngs = []
for i, row in launchesDF.iterrows():
    launchTime = UTCDateTime(row['Date'])
    seismicpngs.append('launch_%s_seismic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    acousticpngs.append('launch_%s_acoustic.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
    wellpngs.append('launch_%s_well.png' % launchTime.strftime('%Y%m%dT%H%M%S') )
launchesDF['seismicpngs']=seismicpngs
launchesDF['acousticpngs']=acousticpngs
launchesDF['wellpngs']=wellpngs
ldf = launchesDF.copy()
ldf.fillna("",inplace=True)
jsonstr = ldf.to_json(orient='columns')
#from json import loads, dumps
#parsed = loads(jsonDf) # turn json string into dict
#jsonstr = dumps(parsed).replace(None, 'null') # python objects to json string

#parsed = json.loads(r)
#print(dumps(parsed, indent=4))

htmlstr = '''
<!DOCTYPE html>
<html>
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Launch Events</title>

    <!-- Basic style -->
    <style>
        .img-container {
            margin: 20px auto;
            width: 40%%;
        }

        .btn-container {
            text-align: center;
        }

        img {
            width: 600px;
            display: block;
            margin: auto;
            padding-bottom: 20px;
        }

        button {
            outline: none;
            padding: 12px;
            border: none;
            background-color: rgb(25, 121, 211);
            border-radius: 8px;
            color: white;
            cursor: pointer;
        }
    </style>

</head>
<body>
    <!-- HTML code for next and previous button -->
    <div class="img-container">
'''
htmlstr += f"        <h1 id=\"pageTitle\">{seismicpngs[0]}</h1>"
htmlstr += f"<img src=\"{seismicpngs[0]}\" class=\"imageSeismic\">"
htmlstr += '''
        <div class="btn-container">
            <button onclick="previous()">Previous</button>
            <button onclick="next()">Next</button>
        </div>
    </div>

    <!-- Javascript code for next and previous button -->
    <script>
    '''

#htmlstr += f"    const jsonObj = JSON.parse({jsonstr});"
htmlstr += f"    const jsonObj = {jsonstr};"
#htmlstr += f"    const jsonObj = {parsed};"
#htmlstr +=  "    console.log(JSON.stringify(jsonObj));"
htmlstr += f"    const seismicpngs = {seismicpngs};"
htmlstr += """

        console.log(seismicpngs);
        console.log(jsonObj['SLC']);
        let result = Object.keys(jsonObj).map(function (key) {
            return jsonObj[key];
        });
        console.log(result);
        
        let imageSeismic = document.querySelector('.imageSeismic');
        var pageTitle = document.getElementById('pageTitle');
    
        let i = 0;

        function next() {
            if (i >= seismicpngs.length - 1) {
                return false;
            }
            i++;
            imageSeismic.setAttribute('src', seismicpngs[i]);
            //pageTitle.textContent = 'seismicpngs[i]';
            document.getElementById('pageTitle').innerHTML = seismicpngs[i];
        }
        function previous() {
            if (i <= 0) {
                return false;
            }
            i--;
            imageSeismic.setAttribute('src', seismicpngs[i]);
            //pageTitle.textContent = 'seismicpngs[i]';
            document.getElementById('pageTitle').innerHTML = seismicpngs[i];
        }

    </script>

</body>

</html>
"""

with open(os.path.join(HTML_DIR,"launches2.html"), "w") as text_file:
    text_file.write(htmlstr)

C:\Users\thompsong\AppData\Local\Temp\ipykernel_2968\24969.py:15: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  ldf.fillna("",inplace=True)


So currently I am getting my dataframe into some sort of name-value key-value hash in Javascript, but I don't have an easy way to access entire columns. And all I know how to do is take each column as an array, and then index it (as with seismicpngs). Try to find out how to index acousticpngs from Javascript key-value hash.

Also need to make RSAM plots where any zeros are turned into NaN. And then a webpage for availability plots.